# Elasticsearch & Warden
Test for searching data over multiple indices 

In [3]:
from elasticsearch import Elasticsearch

ES = Elasticsearch()

print('Testing connection...')
if ES.ping():
    print('Success!')
else:
    print('No connection...')

Testing connection...
Success!


In [8]:
ES.cluster.get_settings(include_defaults=True)


{'persistent': {},
 'transient': {},
 'defaults': {'cluster': {'max_voting_config_exclusions': '10',
   'auto_shrink_voting_configuration': 'true',
   'election': {'duration': '500ms',
    'initial_timeout': '100ms',
    'max_timeout': '10s',
    'back_off_time': '100ms',
    'strategy': 'supports_voting_only'},
   'no_master_block': 'write',
   'persistent_tasks': {'allocation': {'enable': 'all',
     'recheck_interval': '30s'}},
   'blocks': {'read_only_allow_delete': 'false', 'read_only': 'false'},
   'remote': {'node': {'attr': ''},
    'initial_connect_timeout': '30s',
    'connect': 'true',
    'connections_per_cluster': '3'},
   'follower_lag': {'timeout': '90000ms'},
   'routing': {'use_adaptive_replica_selection': 'true',
    'rebalance': {'enable': 'all'},
    'allocation': {'node_concurrent_incoming_recoveries': '2',
     'node_initial_primaries_recoveries': '4',
     'same_shard': {'host': 'false'},
     'total_shards_per_node': '-1',
     'shard_state': {'reroute': {'prior

In [141]:
from mapping import mappings
settings = {"number_of_shards": 1,
            "max_inner_result_window": 10000}


## Using rollover api to create new indices
Lets create a new index and add a rollover to it so that a new one will be created when meeting certain conditions
https://www.elastic.co/guide/en/elasticsearch/reference/master/indices-rollover-index.html

In [142]:
ES.indices.delete(index='_all')
rollover_index = 'my_rollover_index-000001'
rollover_alias = 'my_rollover_alias'
request_body = {'mappings': mappings, 'settings': settings}
ES.indices.create(index=rollover_index, body=request_body)

ES.indices.put_alias(name=rollover_alias, index=rollover_index)
ES.indices.get_alias(name=rollover_alias)

{'my_rollover_index-000001': {'aliases': {'my_rollover_alias': {}}}}

Let's index document to see what it does

In [288]:
from state import State
import time

new_state = State()
print(ES.index(index=rollover_alias, body=new_state.json()))
time.sleep(0.5)
ES.search(body={'size': 0}, index=rollover_alias)

{'_index': 'my_rollover_index-000007', '_type': '_doc', '_id': 'iPB0enEBjn1hTc4IDHKx', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

Since the rollover is an api, we have to call it to execute the rollover if the conditions are met.
The next section will 

In [144]:
request_body = {'conditions': {'max_docs': 1}}
ES.indices.rollover(alias=rollover_alias, body=request_body)
ES.indices.get_alias(name=rollover_alias)


{'my_rollover_index-000002': {'aliases': {'my_rollover_alias': {}}}}

## Automating rollover
For automatic rollover, we need to add policies to our index 
which will execute automatically the rollover when conditons are met
https://www.elastic.co/guide/en/elasticsearch/reference/master/index-lifecycle-management.html

In [208]:
ES.indices.delete(index='_all')
ES.cluster.put_settings(body={
    "transient" : {
        "indices.lifecycle.poll_interval" : "1s"
    }
})
rollover_index_root = 'my_rollover_index'
rollover_index = f'{rollover_index_root}-000001'
rollover_alias = 'my_rollover_alias'
rollover_policy = 'my_rollover_policy'
rollover_template = 'my_rollover_template'

ES.ilm.put_lifecycle(policy=rollover_policy, body={
    'policy': {
        'phases': {
            'hot': {
                'actions': {
                    'rollover': {
                        'max_docs': 5
                    }
                }
            }
        }
    }
})

{'acknowledged': True}

Lets add a template for our indices.
This template will automatically be applied for each index matching the root name.
That way, subsequent created indices will also inherit the policy
and will be subject to a potential rollover if conditions are met

In [210]:
template_settings = {
    "number_of_shards": 1,
    'number_of_replicas': 1,
    'index.lifecycle.name': rollover_policy,
    'index.lifecycle.rollover_alias': rollover_alias,
    "max_inner_result_window": 10000
}

ES.indices.put_template(name=rollover_template, body={
    'index_patterns': [f'{rollover_index_root}-*'],
    'settings': template_settings
})

{'acknowledged': True}

Lets create an index and link an alias to it. We can either set the alias to 'is_write_index=true', in which case
new indices will be added to the alias and the 

In [211]:
ES.indices.create(index=rollover_index)
ES.indices.put_alias(name=rollover_alias, index=rollover_index)
# ES.indices.update_aliases({
#     'actions': [
#         {'add': {'index': rollover_index, 'alias': rollover_alias, 'is_write_index':True}}
#     ]
# })

{'acknowledged': True}

Let's index document to see what it does

In [263]:
from state import State
new_state = State()
ES.index(index=rollover_alias, body=new_state.json())

{'_index': 'my_rollover_index-000004',
 '_type': '_doc',
 '_id': 'WPBgenEBjn1hTc4I8XJl',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [264]:
ES.search(body={'size': 0}, index=rollover_alias)

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [265]:
ES.search(body={'size': 0}, index=rollover_index)

{'took': 8,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [266]:
ES.indices.get_alias(name=rollover_alias)

{'my_rollover_index-000004': {'aliases': {'my_rollover_alias': {}}}}

In [213]:
ES.ilm.explain_lifecycle(index=rollover_alias)


{'indices': {'my_rollover_index-000001': {'index': 'my_rollover_index-000001',
   'managed': True,
   'policy': 'my_rollover_policy',
   'lifecycle_date_millis': 1586895840875,
   'age': '11.63s',
   'phase': 'hot',
   'phase_time_millis': 1586895841636,
   'action': 'rollover',
   'action_time_millis': 1586895846321,
   'step': 'check-rollover-ready',
   'step_time_millis': 1586895846321,
   'phase_execution': {'policy': 'my_rollover_policy',
    'phase_definition': {'min_age': '0ms',
     'actions': {'rollover': {'max_docs': 5}}},
    'version': 11,
    'modified_date_in_millis': 1586895819914}}}}